# Ag-Analytics® - Elevation Index API 

Local elevation on a field can have a strong correlation with harvest yield. The correlation stems from the drainage dynamics that are present on a given field, with low areas being productive in dry years and high areas being productive in wet years. 

The Elevation Index API supports calculation of a set of elevation indices including field relative elevation, maximum slope, ,local Topographic Position Index(TPI), and local Topographic Roughness Index(TRI) for a given field elevation. 

### Required libraries

In [1]:
import requests
import json
import time
import os
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


### Request Parameter Details

Request URL:  https://ag-analytics.azure-api.net/elevation-index

1) __raster__ (string): Local path to the elevation raster the index will be calculated from

2) __Shape__ (Geojson String): Geojson representation of area of interest boundary.
       
3) __index_list__ (string): The elevation index of interest to retrieve.
    
    i. Relative_Elevation: Derived by calculating the z-score of every point in the field
    
    ii. Slop: The maximum slope of each point on the field is calculated by taking the gradient of the elevation for each point.
    
    iii.TPI: The topographic position index is a method of measuring the elevation of a given point with respect to a surrounding neighborhood of surrounding points. This index indicates the local extrema,or points of maximum and minimum elevation on a sub-field level. 
    
    iV.TRI:  The topographic Roughness index express the amount of elevation difference between adjacent points on local field

4) __Outside_Radius(optional)__: Outer radius of search area of each point. Used in TPI only.

        (Ex:Outside_Radius =11 the outer search radius is 11 * cellsize. When cellsize is 0.001 of the elevation raster, the radius is about 11 * 8 = 88 meter

5) __Inside_Radius(optional)__: Inner radius of search area. Used in TPI only.

6) __Search_Radius(optional)__: Radius of computational box. search_radius = 1 => 3x3 box, 2 => 5x5 box, etc. Used in TRI only



### API Function

In [2]:
def ElevationIndex(values, headers, files = None):
    try:
        url = 'https://ag-analytics.azure-api.net/elevation-index'
        
        if files == None:
            response = requests.post(url, json=values, headers=headers).json()
        else:
            response = requests.post(url, files = files, data = values).json()
        #resp = json.loads(response)
        print(response)
        
        return response
    
    except Exception as e:
        print(e)
        raise e

### Request with GeoJSON shape.

In [24]:
# Request as GeoJSON.
values = {'Shape': '{\"type\":\"Feature\",\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-76.5907145, 42.443918], [-76.5898132, 42.4224745], [-76.5699863, 42.4230447], [-76.5710592, 42.4443296], [-76.5907145, 42.443918]]]},\"properties\":{\"OBJECTID\":4944402,\"CALCACRES\":46.15999985,\"CALCACRES2\":null},\"id\":4944402}',
            'index_list':"['Relative_Elevation']",
            'Outside_Radius' :'5',
            'Inside_Radius':'3',
            'Search_Radius' :'2',
            'Legend_Range': '10'
}

headers={'Content-Type':'application/json'}


In [25]:
IndexResponse = ElevationIndex(values, headers)

{'Relative_Elevation': 'raster_RelativeElevation_20200624160129308329.tif', 'features': [[{'attributes': {'CellSize': [9.259259269220297e-05, -9.259259269220297e-05], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-76.5907145, 42.42238515553195, -76.57006635182964, 42.4443296', 'Legend': [{'Area': '10.0 %', 'Count': 4906, 'CountAllPixels': 49060, 'Max': -1.4202146530151367, 'Mean': -2.046072244644165, 'Min': -2.6719298362731934, 'color': '#4d76d5'}, {'Area': '10.0 %', 'Count': 4906, 'CountAllPixels': 49060, 'Max': -0.9852170109748839, 'Mean': -1.2027158319950102, 'Min': -1.4202146530151367, 'color': '#738bbd'}, {'Area': '10.0 %', 'Count': 4906, 'CountAllPixels': 49060, 'Max': -0.600157243013382, 'Mean': -0.7926871269941329, 'Min': -0.9852170109748839, 'color': '#99a1a6'}, {'Area': '10.0 %', 'Cou

### Request using GeoTIFF DEM Raster

In [26]:
#Path to raster in your local machine
rasterpath = r"<Path to DEM Raster>"

files = [('elevation_raster',(open(rasterpath, 'rb')))]

#Parameters to create index
values = {'index_list':"['Relative_Elevation']",
            'Outside_Radius' :'5',
            'Inside_Radius':'3',
            'Search_Radius' :'2',
            'Legend_Range': '10'
}

# Basic Header Pattern.
headers={'Content-Type':'multipart/form-data'}

# Header for using a subscription key.
# headers={'content-type': "application/json",'Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

### Calling API Function and Displaying Response

In [27]:
IndexResponse = ElevationIndex(values, headers, files)

{'Relative_Elevation': 'raster_RelativeElevation_20200624160159048361.tif', 'features': [[{'attributes': {'CellSize': [0.0001, -0.0001], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-76.5907145, 42.4224296, -76.5701145, 42.4443296', 'Legend': [{'Area': '10.0 %', 'Count': 4209, 'CountAllPixels': 42082, 'Max': -1.417577087879181, 'Mean': -2.0448013842105865, 'Min': -2.672025680541992, 'color': '#4d76d5'}, {'Area': '10.0 %', 'Count': 4208, 'CountAllPixels': 42082, 'Max': -0.9816803932189941, 'Mean': -1.1996287405490875, 'Min': -1.417577087879181, 'color': '#738bbd'}, {'Area': '10.0 %', 'Count': 4208, 'CountAllPixels': 42082, 'Max': -0.5979112744331361, 'Mean': -0.7897958338260651, 'Min': -0.9816803932189941, 'color': '#99a1a6'}, {'Area': '10.0 %', 'Count': 4208, 'CountAllPixels': 42082, 'Max': -0

### Output Index Summary

In [28]:
df=defaultdict(list)
for value in IndexResponse['features']:
    attributes = value[0]['attributes']
    df['Index'].append(attributes['Product'])
    df['Max'].append(float(attributes['Max']))
    df['Mean'].append(float(attributes['Mean']))
    df['Min'].append(float(attributes['Min']))
    df['pngb64'].append(attributes['pngb64'])

indexdf=pd.DataFrame.from_dict(df)
indexdf

,Index,Max,Mean,Min,pngb64
0,Relative_Elevation,1.493403,9.949650e-07,-2.672026,"data:image/png;base64, iVBORw0KGgoAAAANSUhEUgA..."


### Displaying Index Image

In [29]:
#####################################
# Select one index to display. Must match one of the following: 
# ['Slope', 'Relative_Elevation', 'TPI', 'TRI']
Index = 'Relative_Elevation'
######################################

In [30]:
#Select one index
Index = Index

#Image Legend
df=defaultdict(list)
for product in IndexResponse['features']:
    if Index == product[0]['attributes']['Product']:
        Legend = product[0]['attributes']['Legend']
for val in Legend:
    df['Max'].append(float(val['Max']))
    df['Mean'].append(float(val['Mean']))
    df['Min'].append(float(val['Min']))
    df['Color'].append(val['color'])
    
legenddf = pd.DataFrame.from_dict(df)    
legenddf.style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])

,Max,Mean,Min,Color
0,-1.417577,-2.044801,-2.672026,#4d76d5
1,-0.981680,-1.199629,-1.417577,#738bbd
2,-0.597911,-0.789796,-0.981680,#99a1a6
3,-0.255589,-0.426750,-0.597911,#bfb68f
4,0.104345,-0.075622,-0.255589,#e5cc78
5,0.485810,0.295078,0.104345,#e5b470
6,0.737060,0.611435,0.485810,#e69c69
7,1.015710,0.876385,0.737060,#e68462
8,1.248521,1.132115,1.015710,#e76c5b
9,1.493403,1.370962,1.248521,#e85454


In [31]:
IndexImage = indexdf.loc[indexdf['Index'] == Index,'pngb64'].iloc[0]
Image(url = IndexImage, width= 500, height =500)

# GET Request

In [10]:
# Specify local path where file will be downloaded.
local_path = r"<Path to local directory>" # E.g., r"C:\Users\John_Doe\Documents\rasters"

# Create GET request payload.
Index = 'Relative_Elevation'
values = {'filenames': [IndexResponse[Index],]}
print(values)

{'filenames': ['raster_RelativeElevation_20200612135833933444.tif']}


In [11]:
def Elevation_Index_get(values, local_path):
    try:
        url = 'https://ag-analytics.azure-api.net/elevation-index'
    
        download_path = os.path.join(local_path, values['filenames'][0])
        print(values)
     
        response = requests.get(url, params=values)
        open(download_path, 'wb').write(response.content)
        
        print(response.url)

        return response
    
    except Exception as e:
        print(e)
        raise e

In [12]:
Elevation_Index_get(values, local_path)

{'filenames': ['raster_RelativeElevation_20200612135833933444.tif']}
https://ag-analytics.azure-api.net/elevation-index?filenames=raster_RelativeElevation_20200612135833933444.tif


<Response [200]>